In [ ]:
# 导入必要的库
import cv2 as cv
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.pyplot as plt
import time

from grayscale import *
from binarization import *
from sharpening import *
from filter import *
from geometric_transform import *
from region_growing import *
from dft import *
from morphology_process import *
from geometry import *
from contour import *
from edge_detect import *
from template_matching import *

In [ ]:
# 图像显示函数
def show_image(image, title='Image'):
    plt.imshow(cv.cvtColor(image, cv.COLOR_BGR2RGB))
    plt.title(title)
    plt.axis('off')
    plt.show()

# 图像处理函数
def process_image(image, method, sub_option, **kwargs):
    gray_image = grayscale(image)
    processed_image = gray_image.copy()    
    method_map = {
        '灰度化': {
            '灰度化': lambda: gray_image,
            '灰度直方图修正': lambda: histogram_equalization(gray_image),
            '自设直方图均衡化': lambda: My_hist_equalization(gray_image),
            '线性变换': lambda: linear_transformation(gray_image),
            '对数变换': lambda: log_transformation(gray_image),
            '指数变换': lambda: exponential_transformation(gray_image)
        },
        '二值化': {
                '固定阈值分割': lambda: fixed_threshold(gray_image, kwargs.get('threshold')),
                '自适应均值阈值分割': lambda: adaptive_mean_threshold(gray_image),
                '自适应高斯阈值分割': lambda: adaptive_gaussian_threshold(gray_image),
                'Otsu自动计算阈值分割': lambda: otsu_threshold(gray_image),
                '自设Otsu': lambda: My_otsu_threshold(gray_image),
            },
        '滤波': {
            '均值滤波': lambda: apply_mean_filter(gray_image, kwargs.get('kernel_size', 5)),
            '高斯滤波': lambda: apply_gaussian_filter(gray_image, kwargs.get('kernel_size', 5), kwargs.get('sigmaX', 1.5)),
            '自设高斯滤波': lambda: My_gaussian_filter(gray_image, kwargs.get('kernel_size', 5), kwargs.get('sigmaX', 1.5)),
            '中值滤波': lambda: apply_median_filter(gray_image, kwargs.get('kernel_size', 5)),
            '双边滤波': lambda: apply_bilateral_filter(gray_image, kwargs.get('d', 9), kwargs.get('sigmaColor', 75), kwargs.get('sigmaSpace', 75))
        },
        '锐化': {
            'Sobel算子': lambda: sobel_sharpen(image),
            'Laplacian算子': lambda: laplacian_sharpen(image)
        },
        '边缘检测':{
            'Sobel边缘检测': lambda: sobel_edge_detect(image),
            'Laplacian边缘检测': lambda: laplacian_edge_detect(image),
            'Canny边缘检测': lambda: canny_edge_detect(image, kwargs.get('low_threshold', 100), kwargs.get('high_threshold', 200))
        },
        '几何变换': {
            '平移': lambda: translate(image, kwargs.get('tx', 0), kwargs.get('ty', 0)),
            '旋转': lambda: rotate(image, kwargs.get('angle', 0)),
            '错切': lambda: shear(image),
            '缩放': lambda: scale(image, kwargs.get('scale_factor', 1.0)),
            '翻转': lambda: flip(image, kwargs.get('flip_direction', 1)),
            '自设仿射变换': lambda: My_affine_transform(image, rotation=kwargs.get('angle', 0), scale=(kwargs.get('scale_factor', 1.0), kwargs.get('scale_factor', 1.0)),
                                                  shear=10, translation=(kwargs.get('tx', 0), kwargs.get('ty', 0))),
        },
        '图像分割': lambda: region_growing(image, seed_point=(400, 100), threshold=8),
        '频域滤波': lambda: frequency_filter(image, sub_option),
        '形态学处理': lambda: morphology_process(image, sub_option),
        '几何形状检测': lambda: geometric_dectect(image, sub_option),
        '轮廓提取': lambda: contours(image, sub_option, kwargs.get('threshold')),
        '模板匹配': lambda: template_match(image, sub_option),
        
    }

    if method in method_map:
        if isinstance(method_map[method], dict):
            processed_image = method_map[method].get(sub_option, lambda: None)()
        else:
            processed_image = method_map[method]()
            
    if method == '灰度化':
        plot_histogram(processed_image)
        
    return processed_image


In [ ]:
# 文件上传控件
file_upload = widgets.FileUpload(
    accept='image/*', multiple=False
)

# 方法选择下拉菜单
method_dropdown = widgets.Dropdown(
    options=['灰度化', '二值化', '滤波', '锐化', '几何变换', '图像分割', 
             '边缘检测', '频域滤波', '形态学处理', '几何形状检测', '轮廓提取', '模板匹配'],  
    value='灰度化', description='处理方法:'
)

# 子选项选择下拉菜单
sub_option_dropdown = widgets.Dropdown(
    options=['灰度化', '灰度直方图修正', '自设直方图均衡化', '线性变换', '对数变换', '指数变换'],
    value='灰度化', description='子选项:'
)

# 处理按钮
process_button = widgets.Button(
    description='处理',
)

# 重置按钮
reset_button = widgets.Button(
    description='重置',
)

# 标题框
title_label = widgets.HTML(value='<b style="font-size:20px; margin-left: 20px;">灰度化: 灰度化</b>')

# 输出区域
output = widgets.Output()

# 文件上传事件处理
def on_file_upload_changed(change):
    global image
    uploaded_files = change['new']
    if isinstance(uploaded_files, tuple):
        uploaded_files = {file.name: {'content': file.content} for file in uploaded_files}
    
    for filename, file_info in uploaded_files.items():
        content = file_info['content']
        image = cv.imdecode(np.frombuffer(content, np.uint8), -1)
        with output:
            clear_output(wait=True)
            show_image(image, 'Original')
            
file_upload.observe(on_file_upload_changed, names='value')

# 更新标题框内容
def update_title():
    title_label.value = f'<b style="font-size:20px; margin-left: 20px;">{method_dropdown.value}: {sub_option_dropdown.value}</b>'

# 处理按钮点击事件处理
def on_process_button_clicked(b):
    with output:
        clear_output(wait=True)

        # 开始计时
        start_time = time.time()

        kwargs = {
            'kernel_size': filter_params.children[0].value,
            'sigmaX': filter_params.children[1].value,
            'd': filter_params.children[2].value,
            'sigmaColor': filter_params.children[3].value,
            'sigmaSpace': filter_params.children[4].value,
            'threshold': threshold_slider.value,
            'low_threshold': edge_detection_params.children[0].value,
            'high_threshold': edge_detection_params.children[1].value,
            'tx': geometry_params.children[0].value,
            'ty': geometry_params.children[1].value,
            'angle': geometry_params.children[2].value,
            'scale_factor': geometry_params.children[3].value,
            'flip_direction': geometry_params.children[4].value,
        }
        processed_image = process_image(image, method_dropdown.value, sub_option_dropdown.value, **kwargs)        

        # 显示原始图像
        fig, axs = plt.subplots(1, 2, figsize=(12, 6))
        axs[0].imshow(cv.cvtColor(image, cv.COLOR_BGR2RGB))
        axs[0].set_title('Original Image')
        axs[0].axis('off')
        
        # 显示处理后的图像
        axs[1].imshow(cv.cvtColor(processed_image, cv.COLOR_BGR2RGB) if len(processed_image.shape) == 3 else processed_image, cmap='gray')
        axs[1].set_title('Processed Image')
        axs[1].axis('off')
        
        plt.show()

        # 结束计时
        end_time = time.time()
        print(f'处理耗时: {end_time - start_time:.2f} 秒')
    update_title()  # 更新标题框内容

process_button.on_click(on_process_button_clicked)

 # 隐藏所有参数控件
def hidden_all_params():
    threshold_slider.layout.visibility = 'hidden'
    for child in filter_params.children:
        child.layout.visibility = 'hidden'
    filter_params.layout.visibility = 'hidden'
    for child in geometry_params.children:
        child.layout.visibility = 'hidden'
    geometry_params.layout.visibility = 'hidden'
    for child in edge_detection_params.children:
        child.layout.visibility = 'hidden'
    edge_detection_params.layout.visibility = 'hidden'

# 重置按钮点击事件处理
def on_reset_button_clicked(b):
    with output:
        clear_output(wait=True)
        show_image(image, 'Original')

    hidden_all_params() # 隐藏所有参数控件
    update_title()  # 重置标题框内容

reset_button.on_click(on_reset_button_clicked)


# 设置控件可见性
def set_visibility(widget, visibility):
    widget.layout.visibility = visibility 

# 方法改变事件处理
def on_method_change(change):
    hidden_all_params()

    method_options = {
        '灰度化': ['灰度化', '灰度直方图修正', '自设直方图均衡化', '线性变换', '对数变换', '指数变换'],
        '二值化': ['固定阈值分割', '自适应均值阈值分割', '自适应高斯阈值分割', 'Otsu自动计算阈值分割', '自设Otsu'],
        '滤波': ['均值滤波', '高斯滤波', '自设高斯滤波', '中值滤波', '双边滤波'],
        '锐化': ['Sobel算子', 'Laplacian算子'],
        '几何变换': ['平移', '旋转', '错切', '缩放', '翻转', '自设仿射变换'],
        '图像分割': ['区域生长'],
        '边缘检测': ['Sobel边缘检测', 'Laplacian边缘检测', 'Canny边缘检测'],
        '频域滤波': ['低通滤波', '高通滤波', '带阻(陷波)滤波'],
        '形态学处理': ['膨胀', '腐蚀', '开运算', '闭运算', '形态学梯度', '顶帽', '黑帽'],
        '几何形状检测': ['轮廓检测'],
        '轮廓提取': ['轮廓绘制'],
        '模板匹配': ['单对象匹配', '多对象匹配', '自设模板匹配'],
    }

    sub_option_dropdown.options = method_options.get(change['new'], [])
    update_title()

    if change['new'] == '二值化' and sub_option_dropdown.value == '固定阈值分割':
        set_visibility(threshold_slider, 'visible') # Threshold
    elif change['new'] == '滤波':
        set_visibility(filter_params, 'visible')
        if sub_option_dropdown.value in ['均值滤波', '高斯滤波', '自设高斯滤波']:
            set_visibility(filter_params.children[0], 'visible')  # Kernel Size
        set_visibility(filter_params.children[1], 'visible' if change['new'] in ['高斯滤波', '自设高斯滤波'] else 'hidden')  # Sigma X
        set_visibility(filter_params.children[2], 'visible' if change['new'] == '双边滤波' else 'hidden')  # Diameter
        set_visibility(filter_params.children[3], 'visible' if change['new'] == '双边滤波' else 'hidden')  # Sigma Color
        set_visibility(filter_params.children[4], 'visible' if change['new'] == '双边滤波' else 'hidden')  # Sigma Space

    elif change['new'] == '几何变换':
        set_visibility(geometry_params, 'visible')
        # 初始化几何变换参数控件的可见性
        for param in geometry_params.children:
            param.layout.visibility = 'hidden'
        
        if sub_option_dropdown.value == '平移':
            set_visibility(geometry_params.children[0], 'visible')  # Translation X
            set_visibility(geometry_params.children[1], 'visible')
        elif sub_option_dropdown.value == '旋转':
            set_visibility(geometry_params.children[2], 'visible')  # Rotation Angle
        elif sub_option_dropdown.value == '缩放':
            set_visibility(geometry_params.children[3], 'visible')  # Scale Factor
        elif sub_option_dropdown.value == '翻转':
            set_visibility(geometry_params.children[4], 'visible')  # Flip Direction
        elif sub_option_dropdown.value == '自设仿射变换':
            for param in geometry_params.children: # Translation X, Translation Y, Rotation Angle, Scale Factor
                set_visibility(param, 'visible')

    elif change['new'] == '边缘检测':
        set_visibility(edge_detection_params, 'visible')
        if sub_option_dropdown.value == 'Canny边缘检测':
            set_visibility(edge_detection_params.children[0], 'visible') # Low Threshold
            set_visibility(edge_detection_params.children[1], 'visible') # High Threshold

    elif change['new'] == '轮廓提取' and sub_option_dropdown.value == '轮廓绘制':
        set_visibility(threshold_slider, 'visible') # Threshold
        
method_dropdown.observe(on_method_change, names='value')

# 子选项改变事件处理
def on_sub_option_change(change):
    hidden_all_params()
    update_title()

    if method_dropdown.value == '二值化' and change['new'] == '固定阈值分割':
        set_visibility(threshold_slider, 'visible')
    elif method_dropdown.value == '滤波':
        set_visibility(filter_params, 'visible')
        set_visibility(filter_params.children[0], 'visible' if change['new'] in ['均值滤波', '高斯滤波', '自设高斯滤波'] else 'hidden')  # Kernel Size
        set_visibility(filter_params.children[1], 'visible' if change['new'] in ['高斯滤波', '自设高斯滤波'] else 'hidden')  # Sigma X
        set_visibility(filter_params.children[2], 'visible' if change['new'] == '双边滤波' else 'hidden')  # Diameter
        set_visibility(filter_params.children[3], 'visible' if change['new'] == '双边滤波' else 'hidden')  # Sigma Color
        set_visibility(filter_params.children[4], 'visible' if change['new'] == '双边滤波' else 'hidden')  # Sigma Space

    elif method_dropdown.value == '几何变换':
        set_visibility(geometry_params, 'visible')
        for param in geometry_params.children:
            set_visibility(param, 'hidden')
        if change['new'] == '平移':
            set_visibility(geometry_params.children[0], 'visible')  # Translation X
            set_visibility(geometry_params.children[1], 'visible')  # Translation Y
        elif change['new'] == '旋转':
            set_visibility(geometry_params.children[2], 'visible')  # Rotation Angle
        elif change['new'] == '缩放':
            set_visibility(geometry_params.children[3], 'visible')  # Scale Factor
        elif change['new'] == '翻转':
            set_visibility(geometry_params.children[4], 'visible')  # Flip Direction
        elif sub_option_dropdown.value == '自设仿射变换':
            for param in geometry_params.children: # Translation X, Translation Y, Rotation Angle, Scale Factor
                set_visibility(param, 'visible')

    elif method_dropdown.value == '边缘检测':
        set_visibility(edge_detection_params, 'visible')
        set_visibility(edge_detection_params.children[0], 'visible' if change['new'] == 'Canny边缘检测' else 'hidden')
        set_visibility(edge_detection_params.children[1], 'visible' if change['new'] == 'Canny边缘检测' else 'hidden')

    elif method_dropdown.value == '轮廓提取' and change['new'] == '轮廓绘制':
        set_visibility(threshold_slider, 'visible')

sub_option_dropdown.observe(on_sub_option_change, names='value')

# 显示控件
button_box = widgets.HBox([process_button, reset_button])
display(file_upload, method_dropdown, sub_option_dropdown, button_box, title_label, threshold_slider, filter_params, geometry_params, edge_detection_params, output)

### 使用 Voila 将 Jupyter Notebook 中的 ipywidgets 交互式界面转换为独立 Web 应用程序

示例命令
假设你的 Notebook 文件位于 /path/to/notebooks 目录下，你可以这样做：

cd /path/to/notebooks

voila interactive_app.ipynb